In [ ]:
!uv add PyMuPDF

In [10]:
import json
import os
from pathlib import PurePath
from pydoc import doc
import re
from turtle import title
from typing import Literal, Optional, Union
from xml.dom.minidom import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from regex import D
from sqlalchemy import false
import fitz # PyMuPDF

file_path = "./docs/국가계약법_시행규칙.pdf"

class HiPDFLoader(PyPDFLoader):
    def __init__(
        self,
        file_path: Union[str, PurePath],
        separators: Optional[list[str]] = ['^제[1-9][0-9]*장 .+$', '^제[1-9][0-9]*조\\(.+\\)$'],
        header_margin=50,
        footer_margin=50) -> None:
        super().__init__(file_path=file_path)
        self.file_path = file_path
        self.header_margin = header_margin
        self.footer_margin = footer_margin
        self.separators = separators
        
    def load(self, split:bool = False) -> list[Document]:
        docs = self.trim_header_footer()
        return self.split(docs) if split else docs
        
    def split(self, docs:list[Document]) -> list[Document]:
        split_doc = []
        
        for doc in docs:
            text_splitter = CharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                separator="\n",
                )
            splits = text_splitter.split_text(doc.page_content)
            split_docs = []
            for j, split in enumerate(splits):
                new_doc = Document()
                new_doc.page_content = split
                new_doc.metadata = {
                    "source": doc.metadata['source'],
                    "page": doc.metadata['page'],
                    "total_pages": doc.metadata['total_pages'],
                    "chunk": j,
                    }
                split_docs.append(new_doc)
            split_doc.extend(split_docs)
        # pages = [doc.page_content for doc in docs]
        # metadatas = [doc.metadata for doc in docs]
        # full_text = '\n\n'.join(pages)
        # title = docs[0].page_content.lstrip()
        # re.compile(p)
        # abbreviation = ''
        return split_doc

    def trim_header_footer(self) -> list[Document]:
        docs = []
        clean_pages = []
        pages = fitz.open(self.file_path)
        title = os.path.basename(self.file_path)
        patterns = [re.compile(p) for p in self.separators]

        for page in pages:
            # 페이지 전체 크기 (width, height)
            page_rect = page.rect
            
            # 제외할 영역을 뺀 '본문 영역' 정의
            # fitz.Rect(x0, y0, x1, y1) -> 좌상단(x0, y0), 우하단(x1, y1)
            content_box = fitz.Rect(
                0,                     # 왼쪽 끝
                self.header_margin,         # 위에서 머리말만큼 아래로
                page_rect.width,       # 오른쪽 끝
                page_rect.height - self.footer_margin  # 아래에서 꼬리말만큼 위로
            )
            
            # 해당 영역(crop box) 내부의 텍스트만 추출
            text = page.get_text('text', clip=content_box).lstrip().rstrip()
            lines = text.lstrip().split('\n')
            page_number = page.number + 1
            
            if page_number == 1:
                title = lines[0] if lines else title
            
            doc = None
            jang = []
            
            for line in lines:
                for p in patterns:
                    match = p.search(line)
                    if match:
                        doc = match.group(0)
                        break
                if doc:
                    break

            document = Document()
            document.page_content = text
            document.metadata = {
                "page": page.number,
                'source': self.file_path,
                'title': title,
                'total_pages': len(pages),
                'jang': jang,
                # 'matchs': [sep for sep in self.separators if re.search(sep, text, re.MULTILINE)]
                }
            docs.append(document)

                    
            
            # p_jang = re.compile(self.separators[0])
            
            # jang_matches = p_jang.search(text, re.MULTILINE)
            # if jang_matches:
            #     for line in lines:
            #         jang_match = p_jang.search(line)
            #         if jang_match:
            #             jang.append(jang_match.group(0))
            #             break
            #     jang_match = p_jang.search(text, re.MULTILINE)
            #     if jang_match:
            #         jang = jang_match.group(0)
                
            # clean_pages.append(text)
            # document = Document()
            # document.page_content = text
            # document.metadata = {
            #     "page": page.number,
            #     'source': self.file_path,
            #     'title': title,
            #     'total_pages': len(pages),
            #     'jang': jang,
            #     # 'matchs': [sep for sep in self.separators if re.search(sep, text, re.MULTILINE)]
            #     }
            # docs.append(document)
            
        return docs


In [13]:
loader = HiPDFLoader(file_path)
docs = loader.load()
# docs
# # --- 본문(first body) 첫줄 추출 로직 ---
# full_text = '\n\n'.join([d.page_content for d in docs]) if docs else ""

# --- 끝 ---
docs[0].metadata
docs[0].page_content

'국가를 당사자로 하는 계약에 관한 법률 시행규칙 ( 약칭: 국가계약법 시행규칙 )\n[시행 2026. 1. 2.] [재정경제부령 제1호, 2026. 1. 2., 타법개정]\n조달청 (규제개혁법무담당관실) 042-724-7277, 7249\n재정경제부 (계약정책과) 044-215-5213, 5217, 5218\n       제1장 총칙\n \n제1조(목적) 이 규칙은 「국가를 당사자로 하는 계약에 관한 법률」 및 동법 시행령에서 위임된 사항과 그 시행에 관하여\n필요한 사항을 규정함을 목적으로 한다. <개정 2005. 9. 8.>\n \n제2조(정의) 이 규칙에서 사용하는 용어의 정의는 다음과 같다. <개정 2003. 12. 12., 2005. 9. 8., 2006. 5. 25.>\n1. “계약담당공무원”이라 함은 세입의 원인이 되는 계약에 관한 사무를 각 중앙관서의 장으로부터 위임 받은 공무원\n, 「국고금관리법」 제22조의 규정에 의한 재무관(대리재무관ㆍ분임재무관 및 대리분임재무관을 포함한다. 이하 같\n다), 「국가를 당사자로 하는 계약에 관한 법률」(이하 “법”이라 한다) 제6조제1항의 규정에 의한 계약관(대리계약\n관ㆍ분임계약관 및 대리분임계약관을 포함한다. 이하 같다) 및 「국고금관리법」 제24조의 규정에 의하여 지출관\n으로부터 자금을 교부받아 지급원인행위를 할 수 있는 관서운영경비출납공무원(대리관서운영경비출납공무원ㆍ\n분임관서운영경비출납공무원 및 대리분임관서운영경비출납공무원을 포함한다. 이하 같다)과 기타 법령에 의하여\n세입세출외의 자금 또는 기금의 출납의 원인이 되는 계약을 담당하는 공무원을 말한다.\n2. “추정금액”이라 함은 공사에 있어서 「국가를 당사자로 하는 계약에 관한 법률 시행령」(이하 “영”이라 한다) 제2조\n제1호에 따른 추정가격에 「부가가치세법」에 따른 부가가치세와 관급재료로 공급될 부분의 가격을 합한 금액을\n말한다.\n \n제3조(적용범위) 각 중앙관서의 장 또는 계약담당공무원은 계약에 관한 사무를 처리함에 있어서 다른 